# Assignment 1

1. Реализовать Блум фильтр с помощью битового массива.
Например, вы можете использовать [Битовые операции](https://wiki.python.org/moin/BitwiseOperators) или библиотеку bitarray.

In [1]:
from math import log, ceil
from random import randint, choice
from string import ascii_lowercase

In [2]:
class Bloom_filter:

    def __init__(self, S, fp):
        self.fp = fp
        self.hashes = [randint(1, 996),
                       randint(1, 996),
                       randint(1, 996)]
        self.arr = [0] * self.get_arr_size(S)
    
    def get_arr_size(self, S):
        return ceil(1 / log(2)**2 * S * log(1/self.fp))
    
    def hash_int(self, x, a=76, b=14, p=733):
        return (a*x + b) % p
        
    def poly_hash(self, s, x, p=997):
        h = 0
        for j in range(len(s)-1, -1, -1):
            h = (h * x + ord(s[j]) + p) % p
        return self.hash_int(h)

    def insert(self, elem):
        indices = [self.poly_hash(elem, num) for num in self.hashes]
        for ind in indices:
            self.arr[ind] = 1

    def lookup(self, elem):
        indices = [self.poly_hash(elem, num) for num in self.hashes]
        for ind in indices:
            if self.arr[ind] == 0:
                return False
        return True

2. Провести численный эксперимент при false postive rate = 0.05, и количестве объектов S = 1 000 000.
Убедится, полученные на семинаре оптимальные параметры фильтра позволяют достичь заявленного false positive rate.
Посчитать  $\frac {\epsilon - \hat \epsilon} \epsilon$, где $\hat \epsilon$ - ваша экспериментальная оценка false positive rate. В качестве объектов используйте строки.

In [3]:
def get_strings(S, min_str, max_str):
    strings = set()
    while len(strings) != S:
        strings.add(''.join([choice(ascii_lowercase) for _ in range(randint(min_str, max_str))]))
    return strings

In [4]:
def get_fp_experimental(S, fp, min_str, max_str):
    bloom_filter = Bloom_filter(S, fp)

    strings_1 = get_strings(S, min_str, max_str)
    strings_2 = get_strings(S, min_str, max_str)
    
    for string in strings_1:
        bloom_filter.insert(string)
        
    fp_experimental = 0
    for string in strings_2:
        if not bloom_filter.lookup(string) and string in strings_1:
            fp_experimental += 1

    return fp_experimental / len(strings_2) * 100

In [5]:
fp = 0.05
S = 1000000
min_str = 0
max_str = 7

fp_experimental = get_fp_experimental(S, fp, min_str, max_str)

print('FP: {}'.format(fp))
print('experimental FP: {}'.format(fp_experimental))
print('(FP - experimental FP) / FP = {}'.format((fp-fp_experimental) / fp))

FP: 0.05
experimental FP: 0.0
(FP - experimental FP) / FP = 1.0
